# Convolutional Autoencoder Architecture (CAE)

In [6]:
import numpy as np
from matplotlib import pyplot as plt
import os

### Data loading

In [7]:
path = "/home/rajesh/Desktop/Datasets/revised_musdbbleed/"
os.listdir(path)

['gt_test.npy', 'gt_train.npy', 'test.npy', 'AIbleed', 'Actual', 'train.npy']

In [8]:
xtrain = np.load(path+'train.npy')
xtest = np.load(path+'test.npy')
ytrain = np.load(path+'gt_train.npy')
ytest = np.load(path+'gt_test.npy')

In [9]:
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((80, 1025, 5168), (80, 1025, 5168), (20, 1025, 5168), (20, 1025, 5168))

### Architecture

In [2]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, \
Flatten, Dropout, BatchNormalization, Reshape
from keras.models import Model

In [3]:
batch_size = 16
epochs = 20
inp = Input(shape=(1025, 5168, 1)) #1025, 5168 2 min

In [4]:
print(inp.shape)

# Conv1 #
x = Conv2D(filters = 1, kernel_size = (15, 15), activation='relu')(inp)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size = (3, 3))(x)
x = Dropout(0.30)(x)
print(x.shape)

# Conv2 #
x = Conv2D(filters = 32, kernel_size = (9, 9), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size = (3, 3))(x)
x = Dropout(0.30)(x)
print(x.shape)

# Conv 3 #
x = Conv2D(filters = 64, kernel_size = (7, 7), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size = (3, 3))(x)
x = Dropout(0.30)(x)
print(x.shape)

# Conv 4 #
x = Conv2D(filters = 128, kernel_size = (5, 5), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size = (3, 3))(x)
x = Dropout(0.30)(x)
print(x.shape)

# Conv 5 #
x = Conv2D(filters = 256, kernel_size = (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size = (3, 3))(x)
encoded = Dropout(0.30)(x)
print(encoded.shape)

x = Flatten()(x)
x = Dense(100, activation='relu')(x)
x = Dense(inp.shape[1] * inp.shape[2] * inp.shape[3], activation='relu')(x)
print(x.shape)
decoded = Reshape((inp.shape[1], inp.shape[2], inp.shape[3]))(x)
print(decoded.shape)

(None, 1025, 5168, 1)


2022-12-22 19:15:56.895576: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-22 19:15:56.895949: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-22 19:15:56.896005: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Rajesh): /proc/driver/nvidia/version does not exist
2022-12-22 19:15:56.905130: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(None, 337, 1718, 1)
(None, 109, 570, 32)
(None, 34, 188, 64)
(None, 10, 61, 128)
(None, 2, 19, 256)


2022-12-22 19:15:57.181804: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2118880000 exceeds 10% of free system memory.
2022-12-22 19:15:57.959363: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2118880000 exceeds 10% of free system memory.
2022-12-22 19:15:58.234642: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2118880000 exceeds 10% of free system memory.


(None, 5297200)
(None, 1025, 5168, 1)


In [5]:
# Autoencoder
deepconv = Model(inp, x)
deepconv.compile(optimizer="adam", loss="binary_crossentropy")
#deepconv.summary()

In [ ]:
deepconv.fit(
    x=xtrain,
    y=ytrain,
    epochs=50,
    batch_size=16,
    shuffle=True,
)